In [4]:
import librosa
import soundfile as sf

import numpy as np
import pandas as pd

import os
from os import listdir
from os.path import isfile, join

from collections import OrderedDict

import json

In [3]:
!pip install librosa


   ---------------------------------------- 0.0/260.1 kB ? eta -:--:--
   ------------ --------------------------- 81.9/260.1 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 260.1/260.1 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.6 kB ? eta -:--:--
   ---------------------------------------- 64.6/64.6 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------ --------------------------- 0.3/1.0 MB 10.2 MB/s eta 0:00:01
   -------------------------------- ------- 0.8/1.0 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/164.9 kB ? eta -:--:--
   ---------------------------------------- 164.9/164.9 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/182.0 kB ? eta -:--:--
   --------------------------------------- 182.0/182.0 kB 10.7 MB/s eta 0:00:00
   ------------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
def extract_features(filepath):
    
    '''
    Source: https://github.com/danz1ka19/Music-Emotion-Recognition/blob/master/Feature-Extraction.py
    Modified to process a single file

        function: extract_features
        input: path to mp3 files
        output: csv file containing features extracted

        This function reads the content in a directory and for each audio file detected
        reads the file and extracts relevant features using librosa library for audio
        signal processing
    '''

    feature_set = {}  # Features

    # Reading audio file
    y, sr = librosa.load(filepath)
    S = np.abs(librosa.stft(y, n_fft=512)) 
    # https://librosa.org/doc/main/generated/librosa.stft.html (set 512 for speech processing)

    # Extracting Features
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=512)
    
    chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)
    
    chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
    melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=512)
    rmse = librosa.feature.rms(y=y)[0]
    cent = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=512)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=512)
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr, n_fft=512)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=512)
    poly_features = librosa.feature.poly_features(S=S, sr=sr, n_fft=512)
    
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    
    zcr = librosa.feature.zero_crossing_rate(y)
    harmonic = librosa.effects.harmonic(y)
    percussive = librosa.effects.percussive(y)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=512)
    mfcc_delta = librosa.feature.delta(mfcc)

    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
    frames_to_time = librosa.frames_to_time(onset_frames[:20], sr=sr)

    # Concatenating Features into one csv and json format
    feature_set['filename'] = filepath  # song name
    feature_set['tempo'] = tempo  # tempo 
    feature_set['total_beats'] = sum(beats)  # beats
    feature_set['average_beats'] = np.average(beats)
    feature_set['chroma_stft_mean'] = np.mean(chroma_stft)  # chroma stft
    feature_set['chroma_stft_var'] = np.var(chroma_stft)
    
    feature_set['chroma_cq_mean'] = np.mean(chroma_cq)  # chroma cq
    feature_set['chroma_cq_var'] = np.var(chroma_cq)
    
    feature_set['chroma_cens_mean'] = np.mean(chroma_cens)  # chroma cens
    feature_set['chroma_cens_var'] = np.var(chroma_cens)
    feature_set['melspectrogram_mean'] = np.mean(melspectrogram)  # melspectrogram
    feature_set['melspectrogram_var'] = np.var(melspectrogram)
    feature_set['mfcc_mean'] = np.mean(mfcc)  # mfcc
    feature_set['mfcc_var'] = np.var(mfcc)
    feature_set['mfcc_delta_mean'] = np.mean(mfcc_delta)  # mfcc delta
    feature_set['mfcc_delta_var'] = np.var(mfcc_delta)
    feature_set['rmse_mean'] = np.mean(rmse)  # rmse
    feature_set['rmse_var'] = np.var(rmse)
    feature_set['cent_mean'] = np.mean(cent)  # cent
    feature_set['cent_var'] = np.var(cent)
    feature_set['spec_bw_mean'] = np.mean(spec_bw)  # spectral bandwidth
    feature_set['spec_bw_var'] = np.var(spec_bw)
    feature_set['contrast_mean'] = np.mean(contrast)  # contrast
    feature_set['contrast_var'] = np.var(contrast)
    feature_set['rolloff_mean'] = np.mean(rolloff)  # rolloff
    feature_set['rolloff_var'] = np.mean(rolloff)
    feature_set['poly_mean'] = np.mean(poly_features)  # poly features
    feature_set['poly_var'] = np.mean(poly_features)
    
    feature_set['tonnetz_mean'] = np.mean(tonnetz)  # tonnetz
    feature_set['tonnetz_var'] = np.var(tonnetz)
    
    feature_set['zcr_mean'] = np.mean(zcr)  # zero crossing rate
    feature_set['zcr_var'] = np.var(zcr)
    feature_set['harm_mean'] = np.mean(harmonic)  # harmonic
    feature_set['harm_var'] = np.var(harmonic)
    feature_set['perc_mean'] = np.mean(percussive)  # percussive
    feature_set['perc_var'] = np.var(percussive)
    feature_set['frame_mean'] = np.mean(frames_to_time)  # frames
    feature_set['frame_var'] = np.var(frames_to_time)
    
    for ix, coeff in enumerate(mfcc):
        feature_set['mfcc' + str(ix) + '_mean'] = coeff.mean()
        feature_set['mfcc' + str(ix) + '_var'] = coeff.var()
    
    return feature_set


In [6]:

new_features_dict = extract_features('record.wav')
df = pd.DataFrame([new_features_dict])
df.to_csv('./new_record.csv', index=False)